In [12]:
import os, sys
import numpy as np
from tools import *
from image_processing import *
from optparse import OptionParser

from model import *

from keras import backend


In [7]:
limit = 11500
backend.set_image_dim_ordering('th')
IMG_SIZE = 224

In [8]:
if os.path.exists('preprocessed_cats.pckl'):

    print ("Loading preprocessed images....")
    preprocessed_cats = restore('preprocessed_cats.pckl')
    preprocessed_dogs = restore('preprocessed_dogs.pckl')
    limit = len(preprocessed_cats)
    
    print ("Loading done.")
else:
    print ("Preprocessing....")
    # cats : 0
    print ("Preprocessing cats images:")
    preprocessed_cats = preprocess_imgs('train/cats', IMG_SIZE, limit = limit)
    store(preprocessed_cats, 'preprocessed_cats.pckl')

    # dogs : 1
    print ("Preprocessing dogs images:")
    preprocessed_dogs = preprocess_imgs('train/dogs', IMG_SIZE, limit = limit)
    store(preprocessed_cats, 'preprocessed_dogs.pckl')
    limit = len(preprocessed_cats)
    

Loading preprocessed images....
Loading done.


In [9]:
# labelling the data
# cats = [1 0] and dogs = [0 1]
zeros = np.zeros((limit, 1))
ones  = np.ones((limit, 1))
Ycats = np.array((ones, zeros))
Ydogs = np.array((zeros, ones))
Y = np.concatenate((Ycats, Ydogs), axis=1).reshape((2*limit, 2, 1)).squeeze()

In [10]:

Xcats = np.array(preprocessed_cats)
Xdogs = np.array(preprocessed_dogs)

X = np.concatenate((Xcats, Xdogs))

X = np.swapaxes(X, 1, 3)

# Construct and test Vgg model

In [11]:
model = create_model(IMG_SIZE)
for layer in model.layers: layer.trainable=False

model.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
model.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
model.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
model.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


In [19]:
classes = get_classes()
print ("Predict a cat: ")
cat_prediction = model.predict(X[15,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print ( cat_prediction, classes[cat_prediction] )

print ("Predict a dog: ")
dog_prediction = model.predict(X[limit+15,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print (dog_prediction, classes[dog_prediction])

Predict a cat: 
(496, u'Christmas_stocking')
Predict a dog: 
(496, u'Christmas_stocking')


# Sepate convolutional layers and dense layers


In [20]:
print ("Total number of layers : {}, only the 6 last layers are dense layers".format(len(model.layers)))

def no_process(x):
    # return x[:, ::-1]
    return x

convolutional_model = Sequential()
dense_model = Sequential()
dense_model.add( Lambda(no_process, input_shape=(512, 7, 7), output_shape=(512, 7, 7) ) )
for i,layer in enumerate(model.layers):
    if i < len(model.layers) - 6:
        convolutional_model.add(layer)
    else:
        dense_model.add(layer)

print ("Total number of convolutional layers : {}".format(len(convolutional_model.layers)))
print ("Total number of dense layers : {}".format(len(dense_model.layers)))

Total number of layers : 38, only the 6 last layers are dense layers
Total number of convolutional layers : 32
Total number of dense layers : 7


In [21]:
convolutional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [17]:
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              4097000   
Total para

In [18]:
# perdict and save the output of the convolutional layers

conv_output = convolutional_model.predict(X)
store(conv_output, 'conv_output.pckl')
print (conv_output.shape)

(23000, 512, 7, 7)


In [15]:
#resore the output if don't want to recompute the cell above

conv_output = restore('conv_output.pckl')
print (conv_output.shape)

(23000, 512, 7, 7)


In [31]:
# check that the output still makes sense
print (conv_output.shape)

print ("Predict a cat: ")
cat_prediction = dense_model.predict(conv_output[49,:,:,:].reshape(1,512, 7, 7)).argmax(axis = 1)[0]
print ( cat_prediction, classes[cat_prediction] )

print ("Predict a dog: ")
dog_prediction = dense_model.predict(conv_output[limit+1950,:,:,:].reshape(1,512, 7, 7)).argmax(axis = 1)[0]
print (dog_prediction, classes[dog_prediction])

(23000, 512, 7, 7)
Predict a cat: 
(0, u'tench')
Predict a dog: 
(0, u'tench')


In [21]:
im = Image.fromarray(np.swapaxes(X[50,:,:], 0 , 2) )
# im = Image.open("train/cats/cat.50.jpg")
im.show()

# Finetune the dense model to detect cats and dogs

In [23]:
print (dense_model.layers[-1])

#dense_model.pop()

for layer in dense_model.layers: layer.trainable=False

In [24]:
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              4097000   
Total para

In [24]:
dense_model.add(Dense(2, activation='softmax', input_shape=(1000,)))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              4097000   
__________

In [25]:
dense_model.compile(
        # loss='binary_crossentropy',
          # optimizer=SGD(lr=10.0),
          optimizer=RMSprop(),
          # optimizer=Adam(lr=0.001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [91]:
dense_model.load_weights("dense_model.h5")

In [26]:
dense_model.fit(conv_output,Y, epochs=1, batch_size=50)
dense_model.save_weights("dense_model.h5")

Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.6932 - acc: 0.4945     


In [ ]:
cat_dog_class = {0: 'cat', 1: 'dog'}

print ("Predict a cat: ")
cat_prediction = dense_model.predict(conv_output[50,:,:].reshape(1,512, 7, 7))
cat_prediction_decision = cat_prediction.argmax(axis = 1)[0]
print ( cat_prediction_decision, cat_dog_class[cat_prediction_decision] )
print (cat_prediction)

print ("Predict a dog: ")
dog_prediction = dense_model.predict(conv_output[limit+1701,:,:].reshape(1,512, 7, 7))
dog_prediction_decision = dog_prediction.argmax(axis = 1)[0]
print (dog_prediction_decision, cat_dog_class[dog_prediction_decision])
print (dog_prediction)

# Finetune the original model to detect cats and dogs

In [72]:
model = create_model(IMG_SIZE)
for layer in model.layers: layer.trainable=False

classes = get_classes()
print ("Predict a cat: ")
cat_prediction = model.predict(X[12,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print ( cat_prediction, classes[cat_prediction] )

print ("Predict a dog: ")
dog_prediction = model.predict(X[1789,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print (dog_prediction, classes[dog_prediction])

Predict a cat: 
(1, u'goldfish')
Predict a dog: 
(285, u'Egyptian_cat')


In [62]:
model.pop()
model.summary()

model.add(Dense(2, activation='sigmoid', input_shape=(1000,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_7 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_40 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 64, 114, 114)      0         
__________

In [63]:
model.compile(
        # loss='binary_crossentropy',
          # optimizer=SGD(lr=10.0),
          optimizer=RMSprop(),
          # optimizer=Adam(lr=0.001),
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [64]:
model.load_weights("model.h5")

In [65]:
model.fit(X,Y, epochs=1, batch_size=50)
model.save_weights("model.h5")

Epoch 1/1
 450/3000 [===>..........................] - ETA: 94s - loss: 0.7363 - acc: 0.8200

KeyboardInterrupt: 

In [209]:
cat_dog_class = {0: 'cat', 1: 'dog'}

print ("Predict a cat: ")
cat_prediction = model.predict(X[50,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print ( cat_prediction, cat_dog_class[cat_prediction] )

print ("Predict a dog: ")
dog_prediction = model.predict(X[1701,:,:].reshape(1,3,224,224)).argmax(axis = 1)[0]
print (dog_prediction, cat_dog_class[dog_prediction])

Predict a cat: 
(0, 'cat')
Predict a dog: 
(1, 'dog')
